In [1]:
import pandas as pd
from sklearn.ensemble import StackingClassifier, BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [2]:
# Данные по соревнованию от Kaggle, состоящие из обучающего и тестового набора
train = pd.read_csv('datasets/train.csv', index_col='id')
test = pd.read_csv('datasets/test.csv', index_col='id')

### Предобработка данных при помощи one-hot кодирования

In [3]:
# One-hot кодирование стобца color
object_train = train.select_dtypes(include='object')['color']
object_test = test.select_dtypes(include='object')['color']
num_train = train.select_dtypes(exclude='object')
num_test = test.select_dtypes(exclude = 'object')
one_hot_train = pd.get_dummies(object_train, dtype='float')
one_hot_test = pd.get_dummies(object_test, dtype='float')

In [4]:
# Объединение закодированных признаков
X_train = num_train.join(one_hot_train)
X_test = num_test.join(one_hot_test)
# Целевой признак
y_train = train['type']

### Применим различные ансамблевые методы классификации

In [5]:
# Стекинг
estimators = [('SVC', SVC(kernel='linear', C=0.4)),
              ('Gns', GaussianNB()),
              ('kNN', KNeighborsClassifier(n_neighbors=17, algorithm='auto'))]
final_estimator = LogisticRegression(C=0.4)
stack = StackingClassifier(estimators=estimators, final_estimator=final_estimator)
stack.fit(X_train, y_train)
stack_prediction = stack.predict(X_test)

In [6]:
# Перевод предсказанных значений в серию pandas и сохранение в csv-файл
predict_stack = pd.Series(stack_prediction, index=test.index).to_frame(name='type').reset_index()
predict_stack.to_csv(r'predicted/predict_monsters_stack.csv', index=False)

In [7]:
# Беггинг
bagging = BaggingClassifier()
params = {'n_estimators': range(10, 50, 10),
          'estimator': [SVC(kernel='linear', C=0.4), LogisticRegression(), GaussianNB()],
          }
grid_bagging = GridSearchCV(bagging, params, cv=5)
grid_bagging.fit(X_train, y_train)
bagging_prediction = grid_bagging.predict(X_test)

In [8]:
grid_bagging.best_params_

{'estimator': LogisticRegression(), 'n_estimators': 30}

In [9]:
# Перевод предсказанных значений в серию pandas и сохранение в csv-файл
predict_bagging = pd.Series(bagging_prediction, index=test.index).to_frame(name='type').reset_index()
predict_bagging.to_csv(r'predicted/predict_monsters_bagging.csv', index=False)

In [10]:
# Лес деревьев (RandomForestClassifier)
parametrs = {'n_estimators': range(1, 202, 50),
             'criterion': ['entropy'],
             'max_depth': range(5, 12, 2),
             'min_samples_split': range(2, 7, 2),
             'max_features': [5]}

random_forest = RandomForestClassifier()
random_bagging = RandomizedSearchCV(random_forest, parametrs, cv=5)
random_bagging.fit(X_train, y_train)
random_prediction = random_bagging.predict(X_test)

In [11]:
# Перевод предсказанных значений в серию pandas и сохранение в csv-файл
predict_forest = pd.Series(random_prediction, index=test.index).to_frame(name='type').reset_index()
predict_forest.to_csv(r'predicted/predict_monsters_forest.csv', index=False)

In [12]:
# Бустинг AdaBoost
ada_boost_params = {'n_estimators': range(100, 501, 100)}
ada_boost = AdaBoostClassifier(estimator=LogisticRegression())
ada_boost_grid = GridSearchCV(ada_boost, ada_boost_params, cv=5)
ada_boost_grid.fit(X_train, y_train)
ada_boost_prediction = ada_boost_grid.predict(X_test)

In [13]:
ada_boost_grid.best_params_

{'n_estimators': 400}

In [14]:
# Перевод предсказанных значений в серию pandas и сохранение в csv-файл
predict_ada_boost = pd.Series(ada_boost_prediction, index=test.index).to_frame(name='type').reset_index()
predict_ada_boost.to_csv(r'predicted/predict_monsters_ada_boost.csv', index=False)

In [15]:
# Бустинг GradientBoosting
gradient_boost_params = {'n_estimators': [100, 300],
                         'max_depth': [1, 3, 5],
                         'learning_rate': [0.1, 0.5]
                         }
gradient_boost = GradientBoostingClassifier()
gradient_boost_grid = GridSearchCV(gradient_boost, gradient_boost_params, cv=5)
gradient_boost_grid.fit(X_train, y_train)
gradient_boost_prediction = gradient_boost_grid.predict(X_test)

In [16]:
gradient_boost_grid.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}

In [17]:
# Перевод предсказанных значений в серию pandas и сохранение в csv-файл
predict_grad_boost = pd.Series(random_prediction, index=test.index).to_frame(name='type').reset_index()
predict_grad_boost.to_csv(r'predicted/predict_monsters_grad_boost.csv', index=False)